# Load Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from scipy.special import boxcox1p
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

# Load Data

In [2]:
players = pd.read_csv('nba_player_stats_2.csv')
team = pd.read_csv('NBA_Team_Stats.csv')

In [8]:
players.head(6)

,rk,player,pos,age,tm,g,gs,mp,fg,fga,...,drb,trb,ast,stl,blk,tov,pf,pts,season,mvp
0,1,Mahmoud Abdul-Rauf,PG,28,SAC,31,0,17.1,3.3,8.8,...,1.0,1.2,1.9,0.5,0.0,0.6,1.0,7.3,1997-98,False
1,2,Tariq Abdul-Wahad,SG,23,SAC,59,16,16.3,2.4,6.1,...,1.2,2.0,0.9,0.6,0.2,1.1,1.4,6.4,1997-98,False
2,3,Shareef Abdur-Rahim,SF,21,VAN,82,82,36.0,8.0,16.4,...,4.3,7.1,2.6,1.1,0.9,3.1,2.5,22.3,1997-98,False
3,4,Cory Alexander,PG,24,TOT,60,22,21.6,2.9,6.7,...,2.2,2.4,3.5,1.2,0.2,1.9,1.6,8.1,1997-98,False
4,4,Cory Alexander,PG,24,SAS,37,3,13.5,1.6,3.9,...,1.1,1.3,1.9,0.7,0.1,1.3,1.4,4.5,1997-98,False
5,4,Cory Alexander,PG,24,DEN,23,19,34.7,4.8,11.1,...,3.9,4.3,6.0,2.0,0.3,2.8,2.0,14.0,1997-98,False


In [4]:
team.head()

,No,Team,G,Min,Pts,Reb,Ast,Stl,Blk,To,...,Oreb,Fgm-a,Pct,3gm-a,Pct.1,Ftm-a,Pct.2,Eff,Deff,Year
0,1,Chicago,103,48.4,96.0,44.1,23.1,8.6,4.3,13.0,...,14.9,36.7-81.7,0.449,3.9-12.0,0.323,18.7-25.2,0.741,111.6,17.5,1997-1998
1,2,Utah,102,48.3,98.6,40.8,24.7,7.6,4.8,14.7,...,11.3,35.9-74.3,0.483,3.1-8.4,0.368,23.8-30.9,0.768,116.3,17.5,1997-1998
2,3,Phoenix,86,48.6,99.3,41.9,25.6,9.2,5.3,14.4,...,12.1,38.2-82.0,0.466,5.2-14.7,0.355,17.7-23.6,0.747,117.1,13.6,1997-1998
3,4,L.A.Lakers,95,48.3,104.8,42.9,24.3,8.7,6.8,14.7,...,13.2,38.0-79.1,0.480,6.1-17.3,0.350,22.8-33.7,0.675,120.8,13.2,1997-1998
4,5,San Antonio,91,48.4,92.5,44.1,21.9,6.2,6.9,15.3,...,11.9,35.1-75.1,0.468,3.7-10.8,0.344,18.5-26.8,0.688,108.0,13.1,1997-1998


## Merge Data

In [6]:
players.tm.unique()

array(['SAC', 'VAN', 'TOT', 'SAS', 'DEN', 'MIL', 'CLE', 'ATL', 'POR',
       'BOS', 'ORL', 'UTA', 'DAL', 'SEA', 'GSW', 'CHH', 'MIA', 'LAC',
       'PHI', 'LAL', 'NJN', 'IND', 'TOR', 'CHI', 'NYK', 'PHO', 'HOU',
       'MIN', 'WAS', 'DET', 'MEM', 'NOH', 'CHA', 'NOK', 'OKC', 'BRK',
       'NOP', 'CHO'], dtype=object)

In [10]:
team.Team.unique()

array(['Chicago', 'Utah', 'Phoenix', 'L.A.Lakers', 'San Antonio',
       'Indiana', 'Miami', 'Seattle', 'Atlanta', 'New York', 'Cleveland',
       'Charlotte', 'Portland', 'Minnesota', 'Washington', 'Detroit',
       'New Jersey', 'Milwaukee', 'Orlando', 'Houston', 'Boston',
       'Philadelphia', 'Sacramento', 'Dallas', 'Vancouver',
       'L.A.Clippers', 'Golden State', 'Toronto', 'Denver', 'Memphis',
       'New Orleans', 'Oklahoma City', 'Brooklyn'], dtype=object)

In [ ]:
team_mapping = {'SAC':'Sacramento', 'VAN':'Vancouver', 'SAS':'San Antonio', 'DEN':'Denver', 'MIL':'Milwaukee', 'CLE':'Cleveland', 'ATL':'Atlanta', 'POR':'Portland',
       'BOS':'Boston', 'ORL':'Orlando', 'UTA':'Utah', 'DAL':'Dallas', 'SEA':'Seattle', 'GSW':'Golden State', 'CHH':'Charlotte', 'MIA':'Miami', 'LAC':'L.A.Clippers',
       'PHI':'Philadelphia', 'LAL':'L.A.Lakers', 'NJN':'New Jersey', 'IND':'Indianapolis', 'TOR':'Toronto', 'CHI':'Chicago', 'NYK':'New York', 'PHO':'Phoenix', 'HOU':'Houston',
       'MIN', 'WAS', 'DET', 'MEM', 'NOH', 'CHA', 'NOK', 'OKC', 'BRK',
       'NOP', 'CHO'}